Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import draw_shapes
import preprocessing
import hows_import
import h5py

# cuda on Nvidia, mps on Mac
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")

Load segmentation map from dataset (HOWS_CL_25)
The dataset should be at 2D_Shape_Completion/HOWS_CL_25

In [ ]:
file = h5py.File("HOWS_CL_25/Images/apple/0/0a2af0e9d0d746169e712d2fddbd61ba/0.hdf5", "r")
segmap = file["segmap"]
np_segmap = np.array(segmap)
binary_image = preprocessing.segmap_to_binary(np_segmap)
surface = preprocessing.get_surface_pixels(binary_image)
sampled_surface = preprocessing.sample_pixels(surface, 50)
# visualize image, surface and sampled surface as images side by side
fig, ax = plt.subplots(1, 3)
ax[0].imshow(binary_image, cmap="gray")
ax[0].set_title("Binary Image")
ax[1].imshow(surface, cmap="gray")
ax[1].set_title("Surface")
ax[2].imshow(sampled_surface, cmap="gray")
ax[2].set_title("Sampled Surface")
#save binary_image to folder segmaps/apple/0/0a2af0e9d0d746169e712d2fddbd61ba/0.png
image_path = "segmaps/apple/0/0a2af0e9d0d746169e712d2fddbd61ba"
os.makedirs(image_path, exist_ok=True)
plt.imsave(image_path + "/0.png", binary_image, cmap="gray")

Would be good to extract the segmentation maps from all images as this is basically the only data we need, no RGB images or depth maps...
Next step: Train a modified U-Net model to predict a shape from a sparse set of sampled surface points.
Code for creating the dataset:

In [ ]:
# Function to process and save segmentation maps and surface samples
def process_and_save(file_path, save_path, prefix):
    file = h5py.File(file_path, "r")
    segmap = file["segmap"]
    np_segmap = np.array(segmap)
    binary_image = preprocessing.segmap_to_binary(np_segmap)
    surface = preprocessing.get_surface_pixels(binary_image)
    
    # Save binary image
    os.makedirs(save_path, exist_ok=True)
    plt.imsave(os.path.join(save_path, f"{prefix}_binary_image.png"), binary_image, cmap="gray")
    
    # Save surface image
    plt.imsave(os.path.join(save_path, f"{prefix}_surface.png"), surface, cmap="gray")
    
    # Save sampled surfaces
    for num_samples in [10, 15, 20, 30, 50]:
        sampled_surface = preprocessing.sample_pixels(surface, num_samples)
        plt.imsave(os.path.join(save_path, f"{prefix}_sampled_surface_{num_samples}.png"), sampled_surface, cmap="gray")

# Walk through all .hdf5 files in the dataset directory
dataset_dir = "HOWS_CL_25/Images"
prefix = 0
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith(".hdf5"):
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, dataset_dir)
            save_path = os.path.join("data", relative_path, str(prefix))
            process_and_save(file_path, save_path, prefix)
            prefix += 1

In [2]:
import shutil

# Define source and destination directories
source_dir = "data"
destination_dir = "data_new"

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Walk through all files in the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if "sampled" not in file:
            # Construct full file path
            file_path = os.path.join(root, file)
            # Copy file to the top-level destination directory
            shutil.copy(file_path, os.path.join(destination_dir, file))
